In [3]:
# Import modules
import numpy as np
import math
from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier 

In [4]:
# Fungsi-fungsi aktivasi
def linear(x, derivative = False):
  if not (derivative):
    return x
  return np.ones_like(x)

def sigmoid(x, derivative = False):
  if not (derivative):
    return 1 / (1 + np.exp(-x))
  return sigmoid(x)*(1 - sigmoid(x))

def relu(x, derivative = False):
  if not (derivative):
    return np.maximum(0, x)
  return np.where(x >= 0, 1, 0)

def softmax(x, derivative = False):
    if not (derivative):
        ex = np.exp(x)
        return ex / np.sum(ex)
    result = np.zeros(x.shape)
    for i in range(x.shape[1]):
        temp = x[:,i].reshape(-1,1)
        resTemp = np.diagflat(temp) - np.dot(temp, temp.T)
        result[:,i] = np.sum(resTemp, axis=1)
    return result

activation_function = {
    "Linear": linear,
    "Sigmoid": sigmoid,
    "ReLU": relu,
    "Softmax": softmax,
}

In [5]:
#Fungsi loss

# Sum of squared errors (Linear, Sigmoid, ReLU)
def sum_of_squared_errors(t, o):
    return 0.5 * np.sum((t - o)**2)

#softmax
def cross_entropy(pk):
    result = (-1)*math.log(pk)
    return result

In [ ]:
class Layer():
  def __init__(self, weight, bias, activation):
    if activation not in ['Linear', 'Sigmoid', 'ReLU', 'Softmax']:
          raise NotImplementedError("Layer activation `%s` is not implemented." 
                                      % activation)
    self.weight = weight
    self.bias = bias
    self.activation = activation

  def forward_propagation(self, input):
    input_T = input.T
    result = np.dot(self.weight, input_T) + self.bias
    return activation_function[self.activation](result)

class NeuralNetwork():
  def __init__(self):
    self.layers = []
  
  def summary(self):
    print("Jumlah layer: ", len(self.layers))
    for i, layer in enumerate(self.layers):
      print("============================================================")
      print('Layer {} (Activation: "{}", Units: {})'
      .format(i+1, layer.activation, len(layer.weight)))
      
      print("Weight:")
      print(np.array(layer.weight))
      
      print("Bias:")
      print(np.array(layer.bias))
    print("============================================================")

  def add(self, layer):
    self.layers.append(layer)
  
  def predict(self, input):
    arr = np.array(input)
    if arr.ndim == 1:
      instance = arr
      instance_res = instance
      for layer in self.layers:
        instance_res = layer.forward_propagation(instance_res)
      return instance_res.tolist()

    batch = arr
    batch_res = []
    for instance in batch:
      instance_res = instance
      for layer in self.layers:
        instance_res = layer.forward_propagation(instance_res)
      
      batch_res.append(instance_res.tolist())
    
    return batch_res
        
  def load_file(self, filename):
    '''
    File format
    <depth>
    <units> <activation function>
    <weight0> 
    <bias0>
    '''
    with open(filename, 'r') as file:
      depth = int(file.readline().strip())
      for i in range (depth):
        line = file.readline().strip().split()
        unit = int(line[0])
        activation = line[1]

        # Weight Matrice
        weight = []
        for j in range(unit):
          weight.append(list(map(float, file.readline().strip().split())))

        # Bias Matrice
        bias = list(map(float, file.readline().strip().split()))
        

        # Add layer
        layer = Layer(weight, bias, activation)
        self.add(layer)
      
      # End of file
    # Close file
    print('File loaded. Model detected')